# Logistic regression

Logistic regression is, somewhat paradoxically, most often used for classification rather than a regression problem. In the case of **binary logistic regression**, the labels of each instance is either 0 or 1, but the regressor predicts a real number between zero and one. This value is typically interpreted as the probability of observing a 1, and then a threshold is chosen to quantize the output to 0 or 1.

## Logistic and logit functions

The **logistic function** is defined as

$$
\sigma(x) = \frac{1}{1+e^{-x}}.
$$

```{figure} ../_static/logistic.png
```

The logistic function takes the form of a smoothed step up from 0 to 1. Its inverse is the **logit function**,

$$
\logit(p) = \ln\left( \frac{p}{1-p} \right).
$$

```{figure} ../_static/logit.png
```

In keeping with interpreting $p$ as probability, $\logit(p)$ is the **log-odds ratio**. For instance, if $p=2/3$, then the odds ratio is $(2/3)/(1/3)=2$ (i.e., 2:1 odds), and $\logit(2/3)=\ln(2)$. 

Logistic regression is the approximation

$$
\logit(p) \approx \bfx^T\bfw,
$$

that is, multilinear regression for the function $\logit(p)$, where $p$ is the probability of the class $y=1$. Hence

$$
p \approx \sigma(\bfx^T\bfw).
$$

## Loss function

At a training observation $(\bfx_i,y_i)$, we know that either $p=0$ or $p=1$. Let $\hat{p}_i$ be the output of the regressor at this observation:

$$
\hat{p}_i  = \sigma(\bfx_i^T\bfw).
$$

The loss function is then

$$
L(\bfw) = -\sum_{i=1}^n \left[ y_i \ln(\hat{p}_i) + (1-y_i) \ln(1-\hat{p}_i) \right].
$$

Note that observation $i$ contributes $-\ln(1-\hat{p}_i)$ if $y_i=0$ and $-\ln(\hat{p}_i)$ if $y_i=1$. Both quantities increase as $\hat{p}_i$ gets farther away from $y_i$. This loss is a special case of **cross-entropy**, a measure of dissimilarity between the probabilities of 1 occurring in the training versus the prediction.

As with other forms of linear regression, the loss function is often regularized using the ridge or LASSO penalty. As we covered earlier, there is a hyperparameter $C$ that emphasizes small $\norm{\bfw}$ as $C\to 0$, and pure regression as $C\to \infty$. 


## Case study: Personal spam filter

We will try logistic regression for a simple spam filter. The data set is based on work and personal emails for one individual. The features are calculated word and character frequencies, as well as the appearance of capital letters.

In [1]:
import pandas as pd
spam = pd.read_csv("spambase.csv")
spam

,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,...,char_freq_%3B,char_freq_%28,char_freq_%5B,char_freq_%21,char_freq_%24,char_freq_%23,capital_run_length_average,capital_run_length_longest,capital_run_length_total,class
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.000,0.000,0.0,0.778,0.000,0.000,3.756,61,278,1
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.000,0.132,0.0,0.372,0.180,0.048,5.114,101,1028,1
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.010,0.143,0.0,0.276,0.184,0.010,9.821,485,2259,1
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.000,0.137,0.0,0.137,0.000,0.000,3.537,40,191,1
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.000,0.135,0.0,0.135,0.000,0.000,3.537,40,191,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4596,0.31,0.00,0.62,0.0,0.00,0.31,0.00,0.00,0.00,0.00,...,0.000,0.232,0.0,0.000,0.000,0.000,1.142,3,88,0
4597,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.000,0.000,0.0,0.353,0.000,0.000,1.555,4,14,0
4598,0.30,0.00,0.30,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.102,0.718,0.0,0.000,0.000,0.000,1.404,6,118,0
4599,0.96,0.00,0.00,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.000,0.057,0.0,0.000,0.000,0.000,1.147,5,78,0


We'll create a feature matrix and label vector, and split into train/test sets.

In [2]:
X = spam.drop("class",axis="columns")
y = spam["class"]

from sklearn.model_selection import train_test_split
X_tr,X_te,y_tr,y_te = train_test_split(X,y,test_size=0.2,shuffle=True,random_state=1)

When using norm-based regularization, it's good practice to standardize the variables, so we will prepare to set up a pipeline.

In [3]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

First we use a large value of $C$ to emphasize the regressive loss rather than the regularization penalty. (The default regularization norm is the 2-norm.) It's not required to select a solver, but we choose one here that is reliable for small data sets.

In [4]:
from sklearn.linear_model import LogisticRegression
logr = LogisticRegression(C=100,solver="liblinear")
pipe = make_pipeline(StandardScaler(),logr)
pipe.fit(X_tr,y_tr)
pipe.score(X_te,y_te)

0.9337676438653637

Let's look at the most extreme regression coefficients.

In [5]:
pd.Series(logr.coef_[0],index=X.columns).sort_values()

word_freq_george             -24.055873
word_freq_cs                  -8.573934
word_freq_hp                  -3.512677
word_freq_meeting             -1.940974
word_freq_lab                 -1.469316
word_freq_edu                 -1.460705
word_freq_hpl                 -1.181911
word_freq_85                  -1.177344
word_freq_re                  -0.981293
word_freq_conference          -0.934152
word_freq_project             -0.776980
word_freq_pm                  -0.477826
char_freq_%3B                 -0.379451
capital_run_length_average    -0.367801
word_freq_data                -0.346597
word_freq_labs                -0.287406
word_freq_original            -0.271247
char_freq_%5B                 -0.209782
word_freq_address             -0.196341
word_freq_table               -0.183580
word_freq_parts               -0.165103
word_freq_direct              -0.135774
word_freq_will                -0.132897
word_freq_1999                -0.124816
word_freq_make                -0.110128


The word "george" is a strong counter-indicator for spam (remember that this data set comes from an individual), while the presence of "free" or consecutive capital letters is a strong signal of spam. 

The predictions by the regressor are all either 0 or 1. But we can also see the forecasted probabilities before rounding.

In [6]:
print("classes:")
print(pipe.predict(X_tr.iloc[:5,:]))
print("\nprobabilities:")
print(pipe.predict_proba(X_tr.iloc[:5,:]))

classes:
[0 0 0 0 0]

probabilities:
[[0.53769264 0.46230736]
 [0.99694715 0.00305285]
 [0.63975661 0.36024339]
 [0.996342   0.003658  ]
 [0.93740435 0.06259565]]


The probabilities might be useful, e.g., to make decisions based on the results.

For a validation-based selection of the best regularization parameter, we can use `LogisticRegressionCV`.

In [7]:
from sklearn.linear_model import LogisticRegressionCV
logr = LogisticRegressionCV(Cs=40,cv=5,solver="liblinear")
pipe = make_pipeline(StandardScaler(),logr)
pipe.fit(X_tr,y_tr)

print(f"best C value: {logr.C_[0]:.3g}")
print(f"R2 score: {pipe.score(X_te,y_te):.4f}")

best C value: 21.5
R2 score: 0.9349


## Multiclass case

When there are more than two unique labels possible, logistic regression can be extended through the **one-vs-rest** paradigm. Given $K$ classes, there are $K$ binary regressors fit for the outcomes "class 1/not class 1," "class 2/not class 2," and so on. This gives predictive relative probabilities $q_1,\ldots,q_K$ for the occurrence of each individual class. Since they need not sum to 1, they can be normalized into predicted probabilities via

$$
\hat{p}_i = \frac{q_i}{\sum_{k=1}^K q_k}.
$$

<!-- 
Another way to convert them is by using a **softmax** function:

$$
p_i = \frac{e^{q_i}}{\sum_{k=1}^K e^{q_k}}.
$$

The softmax exaggerates differences between the $q_i$, making the result closer to a "winner takes all" result.
 -->

## Case study: Gas sensor drift

As a multiclass example, we use a data set about gas sensors recording values over long periods of time.

In [8]:
gas = pd.read_csv("gas_drift.csv")
y = gas["Class"]
X = gas.drop("Class",axis="columns")
X_tr,X_te,y_tr,y_te = train_test_split(X,y,test_size=0.2,shuffle=True,random_state=1)

logr = LogisticRegression(solver="liblinear")
pipe = make_pipeline(StandardScaler(),logr)
pipe.fit(X_tr,y_tr)
pipe.score(X_te,y_te)

0.98705966930266

We can now look at predictions of probability for each class.

In [9]:
import pandas as pd
phat = pipe.predict_proba(X)
pd.DataFrame(phat,columns=["Class "+str(i) for i in range(1,7)])

,Class 1,Class 2,Class 3,Class 4,Class 5,Class 6
0,0.063799,7.519237e-03,0.001233,0.571542,0.008976,3.469312e-01
1,0.006095,5.080764e-01,0.023668,0.407596,0.043982,1.058263e-02
2,0.061460,6.384422e-08,0.000002,0.937864,0.000190,4.848255e-04
3,0.000550,1.155326e-05,0.382025,0.532771,0.084641,8.565403e-11
4,0.034819,1.014740e-05,0.000034,0.959959,0.001611,3.567080e-03
...,...,...,...,...,...,...
13905,0.005055,5.448815e-03,0.001287,0.052044,0.001800,9.343653e-01
13906,0.008545,7.510422e-03,0.000853,0.039269,0.001048,9.427741e-01
13907,0.007100,6.657342e-03,0.000892,0.040465,0.001874,9.430110e-01
13908,0.004199,6.163895e-03,0.002197,0.057378,0.001545,9.285169e-01
